## Exercise 0

In [1]:
def github() -> str:

    return "https://github.com/jphopk/ECON481-Homework/blob/main/ECON481-HW6-JustinHopkins.py"

## Exercise 1

In [5]:
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy import inspect
import pandas as pd
from sqlalchemy.orm import Session
import sqlite3

path = 'auctions.db'

class DataBase:
    def __init__(self, loc: str, db_type: str = "sqlite") -> None:
        """Initialize the class and connect to the database"""
        self.loc = loc
        self.db_type = db_type
        self.engine = create_engine(f'{self.db_type}:///{self.loc}')
    def query(self, q: str) -> pd.DataFrame:
        """Run a query against the database and return a DataFrame"""
        with Session(self.engine) as session:
            df = pd.read_sql(q, session.bind)
        return(df)

auctions = DataBase(path)

def std() -> str:
    query = """
    SELECT itemId, 
           SQRT(SUM((bidAmount - avg_bid)*(bidAmount - avg_bid)) / (COUNT(bidAmount) - 1)) AS std
    FROM (
        SELECT itemId, bidAmount, AVG(bidAmount) OVER(PARTITION BY itemId) AS avg_bid
        FROM bids
    )
    GROUP BY itemId
    HAVING (std is not null)
    """
    return query

print(auctions.query(std()))

       itemId       std
0   172998011  0.000000
1   174445924  0.577350
2   174551689  1.830034
3   174767945  8.607399
4   174871788  3.109126
..        ...       ...
82  182571641  7.767700
83  182660924  4.364634
84  182745253  2.884098
85  182883873  0.707107
86  182925212  1.154701

[87 rows x 2 columns]


## Exercise 2

In [10]:
def bidder_spend_frac() -> str:
    query = """
    WITH Sumbidder AS (
        SELECT HighbidderName AS bidderName, SUM(bidAmount) AS total_spend
        FROM (
            SELECT b1.HighbidderName, b1.bidAmount
            FROM bids b1
            JOIN (
                SELECT itemId, MAX(bidTime) AS latest_timestamp
                FROM bids
                GROUP BY itemId
            ) b2 ON b1.itemId = b2.itemId AND b1.bidTime = b2.latest_timestamp
        ) latest_bids
        GROUP BY bidderName
    ), 
    TotalBids AS (
        SELECT bidderName, MAX(bidAmount) AS total_bids
        FROM bids
        GROUP BY bidderName
    )
    SELECT s.bidderName, s.total_spend, t.total_bids, (s.total_spend / t.total_bids) AS spend_frac
    FROM Sumbidder s
    JOIN TotalBids t ON s.bidderName = t.bidderName;
    """
    return query


print(auctions.query(bidder_spend_frac()))

    bidderName  total_spend  total_bids  spend_frac
0       7****1         7.99       16.00    0.499375
1       A****4        12.00       45.00    0.266667
2       A****C        16.99       16.99    1.000000
3       A****a        40.98       40.98    1.000000
4       A****e        79.90       55.00    1.452727
..         ...          ...         ...         ...
147     v****6        30.00       31.00    0.967742
148     v****l        22.00       22.00    1.000000
149     w****5        46.99       46.99    1.000000
150     w****7        19.99       19.99    1.000000
151     y****a        34.99       34.99    1.000000

[152 rows x 4 columns]


## Exercise 3

In [11]:
def min_increment_freq() -> str:
    query = """
    SELECT 
        SUM(CASE 
                WHEN b2.bidAmount = b1.bidAmount + i.bidIncrement THEN 1
                ELSE 0
            END) * 1.0 / COUNT(b2.bidAmount) AS freq
    FROM bids b1
    JOIN bids b2 ON b1.itemId = b2.itemId AND b1.bidAmount < b2.bidAmount
    JOIN items i ON i.itemId = b1.itemId
    WHERE i.isBuyNowUsed = 0
    GROUP BY b1.itemId;
    """
    return query

print(auctions.query(min_increment_freq()))

        freq
0   1.000000
1   0.166667
2   0.076923
3   0.333333
4   1.000000
..       ...
78  0.037037
79  0.333333
80  0.222222
81  1.000000
82  1.000000

[83 rows x 1 columns]


## Exercise 4

In [12]:
def win_perc_by_timestamp() -> str:
    query = """
    WITH AuctionTimes AS (
        SELECT
            itemId, MIN(bidTime) AS startTime, MAX(bidTime) AS endTime
        FROM bids
        GROUP BY itemId
    ),
    BidsWithNormalizedTime AS (
        SELECT
            b.itemId, b.bidTime, b.bidAmount, t.startTime, t.endTime,
            CASE
                WHEN (julianday(t.endTime) - julianday(b.bidTime)) / (julianday(t.endTime) - julianday(t.startTime)) < 0.1 THEN 1
                WHEN (julianday(t.endTime) - julianday(b.bidTime)) / (julianday(t.endTime) - julianday(t.startTime)) < 0.2 THEN 2
                WHEN (julianday(t.endTime) - julianday(b.bidTime)) / (julianday(t.endTime) - julianday(t.startTime)) < 0.3 THEN 3
                WHEN (julianday(t.endTime) - julianday(b.bidTime)) / (julianday(t.endTime) - julianday(t.startTime)) < 0.4 THEN 4
                WHEN (julianday(t.endTime) - julianday(b.bidTime)) / (julianday(t.endTime) - julianday(t.startTime)) < 0.5 THEN 5
                WHEN (julianday(t.endTime) - julianday(b.bidTime)) / (julianday(t.endTime) - julianday(t.startTime)) < 0.6 THEN 6
                WHEN (julianday(t.endTime) - julianday(b.bidTime)) / (julianday(t.endTime) - julianday(t.startTime)) < 0.7 THEN 7
                WHEN (julianday(t.endTime) - julianday(b.bidTime)) / (julianday(t.endTime) - julianday(t.startTime)) < 0.8 THEN 8
                WHEN (julianday(t.endTime) - julianday(b.bidTime)) / (julianday(t.endTime) - julianday(t.startTime)) < 0.9 THEN 9
                ELSE 10
            END AS timestamp_bin
        FROM bids b
        JOIN AuctionTimes t ON b.itemId = t.itemId
    ),
    WinningBids AS (
        SELECT itemId, MAX(bidAmount) AS highestBidAmount
        FROM bids
        GROUP BY itemId
    )
    SELECT
        n.timestamp_bin,
        100.0 * SUM(CASE WHEN n.bidAmount = w.highestBidAmount THEN 1 ELSE 0 END) / COUNT(*) AS win_perc
    FROM BidsWithNormalizedTime n
    JOIN WinningBids w ON n.itemId = w.itemId
    GROUP BY n.timestamp_bin
    """
    return query
    
print(auctions.query(win_perc_by_timestamp()))

   timestamp_bin   win_perc
0              1  37.552743
1              2  10.000000
2              3   0.000000
3              4  11.764706
4              5  15.384615
5              6   0.000000
6              7   0.000000
7              8   0.000000
8              9  11.111111
9             10  49.732620
